## Part 1: Pulse Rate Algorithm

### Contents
Fill out this notebook as part of your final project submission.

**You will have to complete both the Code and Project Write-up sections.**
- The [Code](#Code) is where you will write a **pulse rate algorithm** and already includes the starter code.
   - Imports - These are the imports needed for Part 1 of the final project. 
     - [glob](https://docs.python.org/3/library/glob.html)
     - [numpy](https://numpy.org/)
     - [scipy](https://www.scipy.org/)
- The [Project Write-up](#Project-Write-up) to describe why you wrote the algorithm for the specific case.


### Dataset
You will be using the **Troika**[1] dataset to build your algorithm. Find the dataset under `datasets/troika/training_data`. The `README` in that folder will tell you how to interpret the data. The starter code contains a function to help load these files.

1. Zhilin Zhang, Zhouyue Pi, Benyuan Liu, ‘‘TROIKA: A General Framework for Heart Rate Monitoring Using Wrist-Type Photoplethysmographic Signals During Intensive Physical Exercise,’’IEEE Trans. on Biomedical Engineering, vol. 62, no. 2, pp. 522-531, February 2015. Link

-----

### Code

In [1]:
import glob

import numpy as np
import scipy as sp
from scipy import signal
import scipy.io as sio
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold

In [2]:
def LoadTroikaDataset():
    """
    Retrieve the .mat filenames for the troika dataset.

    Review the README in ./datasets/troika/ to understand the organization of the .mat files.

    Returns:
        data_fls: Names of the .mat files that contain signal data
        ref_fls: Names of the .mat files that contain reference data
        <data_fls> and <ref_fls> are ordered correspondingly, so that ref_fls[5] is the
            reference data for data_fls[5], etc...
    """
    data_dir = "./datasets/troika/training_data"
    data_fls = sorted(glob.glob(data_dir + "/DATA_*.mat"))
    ref_fls = sorted(glob.glob(data_dir + "/REF_*.mat"))
    return data_fls, ref_fls

def LoadTroikaDataFile(data_fl):
    """
    Loads and extracts signals from a troika data file.

    Usage:
        data_fls, ref_fls = LoadTroikaDataset()
        ppg, accx, accy, accz = LoadTroikaDataFile(data_fls[0])

    Args:
        data_fl: (str) filepath to a troika .mat file.

    Returns:
        numpy arrays for ppg, accx, accy, accz signals.
    """
    data = sp.io.loadmat(data_fl)['sig']
    return data[2:]


def AggregateErrorMetric(pr_errors, confidence_est):
    """
    Computes an aggregate error metric based on confidence estimates.

    Computes the MAE at 90% availability.

    Args:
        pr_errors: a numpy array of errors between pulse rate estimates and corresponding
            reference heart rates.
        confidence_est: a numpy array of confidence estimates for each pulse rate
            error.

    Returns:
        the MAE at 90% availability
    """
    # Higher confidence means a better estimate. The best 90% of the estimates
    #    are above the 10th percentile confidence.
    percentile90_confidence = np.percentile(confidence_est, 10)

    # Find the errors of the best pulse rate estimates
    best_estimates = pr_errors[confidence_est >= percentile90_confidence]

    # Return the mean absolute error
    return np.mean(np.abs(best_estimates))

In [3]:
SAMPLING_FREQUENCY = 125
BAND_PASS = (40/60, 240/60)
WIN_SHIFT = 2
WIN_LENGTH = 10

def bandpass_filter(data, band_pass=BAND_PASS, fs=SAMPLING_FREQUENCY):
    """
    Bandpass filter the signal data

    Args:
        data: data to be filtered
        band_pass: critical frequency range
        fs: sampling frequency

    Returns:
        filtered signal
    """
    b, a = signal.butter(3, band_pass, btype='bandpass', fs=fs)
    return signal.filtfilt(b, a, data)

def get_indices(sig_len, ref_len, fs=SAMPLING_FREQUENCY, win_shift=WIN_SHIFT, win_length=WIN_LENGTH):
    """
    Returns the start and end indices of signal data

    Args:
        sig_len: data to be filtered
        ref_len: critical frequency range
        fs: sampling frequency
        win_shift: default = 2
        win_length: default = 10

    Returns:
        start, end indices
    """
    start = (np.cumsum(np.ones(min(sig_len, ref_len)) * fs * win_shift) - fs * win_shift).astype(int)
    end = start + fs * win_length
    return start, end

def create_feature(data, band_pass=BAND_PASS, fs=SAMPLING_FREQUENCY):
    """
    Create features
    """
    (ppg, accx, accy, accz) = data
    n = len(ppg) * 4

    freqs = np.fft.rfftfreq(n, 1/fs)
    fft = np.abs(np.fft.rfft(ppg,n))

    fft[freqs <= band_pass[0]] = 0.0
    fft[freqs >= band_pass[1]] = 0.0

    acc_mag = np.sqrt(accx**2 + accy**2 + accz**2)
    acc_fft = np.abs(np.fft.rfft(acc_mag, n))
    acc_fft[freqs <= band_pass[0]] = 0.0
    acc_fft[freqs >= band_pass[1]] = 0.0

    ppg_feature = freqs[np.argmax(fft)]
    acc_feature = freqs[np.argmax(acc_fft)]

    return np.array([ppg_feature, acc_feature])

def create_data(data_fls, ref_fls):
    features, labels = [], []

    for data_fl, ref_fl in zip(data_fls, ref_fls):
        (ppg, accx, accy, accz) = LoadTroikaDataFile(data_fl)
        ref = sio.loadmat(ref_fl)['BPM0'].reshape(-1)

        start_indices, end_indices = get_indices(len(accx), len(ref))
        for i, (s, e) in enumerate(zip(start_indices, end_indices)):
            f_ppg = bandpass_filter(ppg[s:e])
            f_accx = bandpass_filter(accx[s:e])
            f_accy = bandpass_filter(accy[s:e])
            f_accz = bandpass_filter(accz[s:e])

            features.append(create_feature((f_ppg, f_accx, f_accy, f_accz)))
            labels.append(ref[i])

    features, labels = np.array(features), np.array(labels)
    return features, labels

def train_regression_model(data_fls, ref_fls):
    """
    Train Regressor model

    Returns:
         trained model
    """
    features, labels = create_data(data_fls, ref_fls)
    model = RandomForestRegressor(n_estimators = 400, max_depth = 20)
    for train_idx, test_idx in KFold(n_splits=5).split(features, labels):
        x_train, y_train = features[train_idx], labels[train_idx]
        x_test, y_test = features[test_idx], labels[test_idx]
        model.fit(x_train, y_train)
    return model

def RunPulseRateAlgorithm(data_fl, ref_fl, model, fs=SAMPLING_FREQUENCY):
    # Compute pulse rate estimates and estimation confidence.
    errs = []
    confs = []

    ppg, accx, accy, accz = LoadTroikaDataFile(data_fl)
    refs = sio.loadmat(ref_fl)['BPM0'].reshape(-1)

    start_indices, end_indices = get_indices(len(accx), len(refs))
    for i, (s, e) in enumerate(zip(start_indices, end_indices)):
        ref = refs[i]
        w_ppg = bandpass_filter(ppg[s:e])
        w_accx = bandpass_filter(accx[s:e])
        w_accy = bandpass_filter(accy[s:e])
        w_accz = bandpass_filter(accz[s:e])

        feature = create_feature((w_ppg, w_accx, w_accy, w_accz))
        pred = model.predict(np.reshape(feature, (1, -1)))[0]

        # calculate confidence
        n = len(w_ppg) * 4
        freqs = np.fft.rfftfreq(n, 1/fs)
        fft = np.abs(np.fft.rfft(w_ppg,n))
        fft[freqs <= 40/60.0] = 0.0
        fft[freqs >= 240/60.0] = 0.0

        est_fs = pred / 55.0
        fs_win = 30  / 60.0
        fs_win_e = (freqs >= est_fs - fs_win) & (freqs <= est_fs +fs_win)
        conf = np.sum(fft[fs_win_e])/np.sum(fft)

        errs.append(np.abs(pred - ref))
        confs.append(conf)

    # Return per-estimate mean absolute error and confidence as a 2-tuple of numpy arrays.
    errors, confidence = np.array(errs), np.array(confs)
    return errors, confidence

In [4]:
def Evaluate():
    """
    Top-level function evaluation function.

    Runs the pulse rate algorithm on the Troika dataset and returns an aggregate error metric.

    Returns:
        Pulse rate error on the Troika dataset. See AggregateErrorMetric.
    """
    # Retrieve dataset files
    data_fls, ref_fls = LoadTroikaDataset()
    model = train_regression_model(data_fls, ref_fls)
    errs, confs = [], []
    for data_fl, ref_fl in zip(data_fls, ref_fls):
        # Run the pulse rate algorithm on each trial in the dataset
        errors, confidence = RunPulseRateAlgorithm(data_fl, ref_fl, model)
        errs.append(errors)
        confs.append(confidence)
        # Compute aggregate error metric
    errs = np.hstack(errs)
    confs = np.hstack(confs)
    return AggregateErrorMetric(errs, confs)

In [5]:
Evaluate()

8.685631699638144

-----
### Project Write-up

Answer the following prompts to demonstrate understanding of the algorithm you wrote for this specific context.

> - **Code Description** - Include details so someone unfamiliar with your project will know how to run your code and use your algorithm.
> - **Data Description** - Describe the dataset that was used to train and test the algorithm. Include its short-comings and what data would be required to build a more complete dataset.
> - **Algorithhm Description** will include the following:
>   - how the algorithm works
>   - the specific aspects of the physiology that it takes advantage of
>   - a describtion of the algorithm outputs
>   - caveats on algorithm outputs 
>   - common failure modes
> - **Algorithm Performance** - Detail how performance was computed (eg. using cross-validation or train-test split) and what metrics were optimized for. Include error metrics that would be relevant to users of your algorithm. Caveat your performance numbers by acknowledging how generalizable they may or may not be on different datasets.

Your write-up goes here...

**Code Description**
This code is used to test performance of Pulse Rate Algorithm on the training test. You can run the `Evaluate` function to get pulse rate error on the Troika dataset

- RunPulseRateAlgorithm: Compute pulse rate estimates and estimation confidence.
- Evaluation:  Runs the pulse rate algorithm on the Troika dataset and returns an aggregate error metric.

**Data Description**

We will be using the Troika dataset to build the algorithm. There are 12 datasets used to evaluate our Pulse Rate Algorithm.
- Each dataset file include PPG signals and 3-axis acceleration signals
- ECG is ground truth

**Algorithm Description**
1. How the algorithm works
- Performs bandpass filtering to make signal in a reasonable range (40-240 BPM)
- The accelerometer data are aggregated in the 3 directions to get aggregated magnitude
- Train a RandomForestRegression.
- Compute pulse rate estimates and estimation confidence

2. The specific aspects of the physiology that it takes advantage of
- Pulse reate is estimated by using PPG signals. The capillaries in the wrist are filled with blood when the ventricles contract. When there are few red blood cells the light from the PPG sensor will be reflected in a large amount. When the blood cells are many, they absorb the light, so the optical detector will detect less light. Changes in the light detected by the optical detector, will produce an oscillating waveform which is the pulse rate.

3. A description of the algorithm outputs
- The estimated frequency (in BPM) and the confidence score of that prediction.

4. Caveats on algorithm outputs
- Results in a weak PPG signal where our algorithm could miss detecting a peak in the frequency domain that represents a pulse rate

5. Common failure modes
- The algorithm may not work in some case of changing heart rate due to amr, hand movements.

**Algorithm Performance**
The performance is calculated using the mean absolute error (MAE) between the estimations and the ground truth signal. The MEA of the algorithm when evaluated on the test set is around 8.7.



-----
### Next Steps
You will now go to **Test Your Algorithm** (back in the Project Classroom) to apply a unit test to confirm that your algorithm met the success criteria. 